# Introduction 
In this code we're going to wok through an extended example that demonstrates how randomization inference works. As noted in the lecture, we are working through the example of a randomized assignment of men and women to eat (or not eat) soybeans and we are measuring the level of estrogen present in each of their bloodstreams (perhaps in parts/million).


# Make Data 

First, we create a grouping variable with two groups, one called "Man", and another called "Woman". 

> Alex apologizes that David couldn't come up with an example that is either more interesting or less gender-binary. 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
df = pd.DataFrame({'group': np.repeat(a=['man', 'woman'], repeats=20)})
df.tail()

To these groups, we assign silly, but schematically heplful *potential outcomes* to treatment and control. We say that, by some chance, we sampled men into our study that had estrogen ppm levels that ranged from 1-20, in perfect increments. Also, what luck, but we sampled women into the study that had estrogen levels that ranged from 51-70. So, the women have on average higher estrogen beginning the study. 

In [ ]:
df['po_control'] = np.concatenate([np.arange(1,21), np.arange(51,71)], axis=None)
df['po_treat'] = df['po_control'] # no effect because potential outcomes are the same 

Because we're going to use this later, let's wrap this *universe* creation into a function. 

In [ ]:
def make_data(effect_size, number_per_group): 
    '''
    This dataframe is going to be hardcoded to have 40 folks, 20 "M" and 20 "F". 
    The only argument is the magnitude of the difference between po_control and _treat
    '''
    
    df = pd.DataFrame({'group': np.repeat(a=['man', 'woman'], repeats=number_per_group)})
    df['po_control'] = np.concatenate(
        [np.arange(1,number_per_group+1), 
         np.arange(1,number_per_group+1)+50],
        axis=None)
    df['po_treat'] = df['po_control'] + effect_size
    
    return(df)

In [ ]:
NUMBER_PER_GROUP=200

df = make_data(effect_size=0, number_per_group=NUMBER_PER_GROUP)
df.tail()

Once the data is built, it is then we can start to consider actually interviening in the world to conduct the experiment. 

# Conduct Experiment 

Per our randomization scheme, we are going to randomly assign the individuals to either: 

- Eat lots of tofu `(treatment == 1)`; or, 
- Not eat lots of tofu `(treatment == 0)`. 

To do this, we write a simple function that will randomly place zeros and ones for the treatment and control. 

In [ ]:
def randomize(number_per_group): 
    urn = np.repeat([0,1], repeats=number_per_group)
    np.random.shuffle(urn)
    return(urn)

With our randomization function in hand, we can now "**run an experiment**" which is going to: 

1. Assign people to receive treatment or control; 
2. If they receive treatment, we will see their potential outcomes to treatment; and, 
3. If they're in control, we will see their potential outcomes to control. 

Functionally, this is simply storing the results of our function randomize in a vector object called treatment, and then recording the correct potential outcome value. 

In [ ]:
def run_experiment(_df, number_per_group, science_table): 
    '''
    This takes an argument number per group, and an argument for
    whether to return the science table (if True) or the observed
    data (if False).
    '''
    _df['treat'] = randomize(number_per_group)
    
    _df['Y'] = np.NaN
    _df.loc[df['treat']==0, 'Y'] = df.loc[df['treat']==0, 'po_control']
    _df.loc[df['treat']==1, 'Y'] = df.loc[df['treat']==1, 'po_treat']

    if science_table==False: 
        _df = _df[['group', 'treat', 'Y']]
    
    elif science_table==True: 
        _df = _df
        
    return(_df)

In [ ]:
df_ = run_experiment(df, NUMBER_PER_GROUP, True)
df_.tail()

Recall that we are setting up an experiment that has **no** effect. As Green and Gerber point out in *Field Experiments* in the case of the sharp-null, we are actually testing against the possiblity that we observe both the potential outcoems! Cool.

Next, we create a vector of realized outcomes, first using the compact notation that Green and Gerber prefer using maths. For those randomized to treamtent, we multiply the potential outcome to treatment time the treatment vector (which is a 1 when they were assigned to treatment), and for those in control, into this vector we assign the potential outcome to control time the quantity `(1 - treatment)` which will be one when they are in the treatment group. 

# Ok so we've got our data set up. Now what? 
Now that we have the data set up, we can begin to examine what the lecture is really about, what is the distribution of ATE that we observe due to the different possibly assignments to treatment and control. A few points to remember: 

1. From last week: The difference in sample means between the treatment and control groups is an unbiased estimator of the ATE. 
2. Also from last week: This does not imply that any one realization of treatment/control assignment is guaranteed to exactly produce that ATE.

This is the entire point of understanding the distribution of the ATE. 

To get here, first lets write another function that will calculate the ATE. 


In [ ]:
def estimate_ate(_df): 
    '''This takes a df with names treat and Y.'''
    res = _df[['treat', 'Y']].groupby('treat').mean()
    ate = res.diff()['Y'][1]
    
    return(ate)

In [ ]:
ate = estimate_ate(_df = df_)
ate

## What gives?!

We created this data such that there is *exactly* zero treament effect, but when we look at this summary of the outcomes, they're not **exactly** the same! *Shouldn't they have been?* 

Note, even more specifically than creating the data so that there was no *average treatment effect* we constructed this so that there was no effect at all -- for every single person, `po_control == po_treat`. 


What happens if you go back to the top and run the experiment again? Are the numbers internally consistent, even if they're producing a result that doesn't seem to make sense? Go back up and look at these again. 

## Question of Understanding 

.

.

.

.

.

.

Why are these values different each time you run your experiment? 

> Answer here. 

## More Work with this 

Lets rewrite our function just slightly so that rather than pulling the results frame, instead we're pulling the estimated average treatment effect from the experiment. This is just going to index into the values, and compute the difference between them. 

In [ ]:
def ri_sim_experiment(_df, number_per_group): 
    _df['treat'] = randomize(number_per_group)
    res = _df[['treat', 'Y']].groupby('treat').mean()
    res = res.diff()
    return(res.iloc[1]['Y'].astype('int'))

In [ ]:
df_.head()

In [ ]:
f'The ate under the sharp null is: {ri_sim_experiment(df_, NUMBER_PER_GROUP)}.'

In [ ]:
f'The ate under the sharp null is: {ri_sim_experiment(df_, NUMBER_PER_GROUP)}.'

In [ ]:
f'The ate under the sharp null is: {ri_sim_experiment(df_, NUMBER_PER_GROUP)}.'

What we're seeing is that there are some slightly different outcomes to different **randomization inference treatment regimes**. 

We can do this, a bunch more times using some simple list comprehension. Specifically, do this 1,000 times to get a sense of the distribution. 


In [ ]:
experiments = [ri_sim_experiment(df_, NUMBER_PER_GROUP) for _ in range(0,1000)]

In [ ]:
experiments[0:10]

In [ ]:
%matplotlib inline

plt.hist(experiments)
plt.axvline(x=ate, color = 'darkorange')
plt.axvline(x = -1 * ate, color = 'darkorange')
plt.title('Histogram of RI ATES')
plt.xlabel('RI ATE')

As it turns out, that was pretty similar to what we saw in our draw! 

In [ ]:
ate

In fact, what we've got here is pretty likely to turn up by chance. 

Following David's statment, we can draw a probability of seeing an ATE of a given size under the repeated randomization regime. 

For how many of these randomization inference loops was the *randomization inference* ate more extreme than the observed ate when you **actually** ran the experiment? 

In [ ]:
def p_value(ate, ri_dist): 
    test_vector = np.abs(ri_dist) > np.abs(ate)
    test_vector = test_vector.astype('int')
    
    p_value = test_vector.mean()

    return(p_value)

In [ ]:
p_value(ate=ate, ri_dist=experiments)

In [ ]:
# If you want to see each of the sides of this happen; you might consider 
# reading this version of the same function. 
#
# def p_value(ate, ri_dist): 
#     if np.sign(ate) == -1: 
#         negative_vals = (ri_dist < ate).astype('int').mean()
#         positive_vals = (ri_dist > -1 * ate).astype('int').mean()
#     elif np.sign(ate) == 1: 
#         negative_vals = (ri_dist < -1 * ate).astype('int').mean()
#         positive_vals = (ri_dist > ate).astype('int').mean()
#     pvalue = negative_vals + positive_vals
#    
#     return(pvalue)
#
# p_value(ate=ate, ri_dist=experiments)

In [ ]:
experiments_no_effect = experiments

# Simulate an Experiment with a Large Effect 

We have seen that when there is no effect, our Randomization Inference regime does a good job at assigning a high probability of observing an effect size equal to or larger than the ATE we calculate from our particular randomization. 

Now, let's show that when there is a big effect, our Randomization Inference regime does a good job at assigning a low probability of observing an effect size equal to or larger that the ATE we calculate in our regression. 


In [ ]:
df = make_data(effect_size=25, number_per_group=NUMBER_PER_GROUP)

In [ ]:
df = run_experiment(df, NUMBER_PER_GROUP, False)
df.iloc[:10, ]

If that is the data that is generated from our actual experiment, how do we generate the average treatment effect? 

In [ ]:
ate = estimate_ate(df)
ate

If that is the estimated ate, what do we get for a histogram of randomization inference? 

In [ ]:
f'The ate under the assumption of the sharp null is {ri_sim_experiment(df, NUMBER_PER_GROUP)}.'

In [ ]:
f'The ate under the assumption of the sharp null is {ri_sim_experiment(df, NUMBER_PER_GROUP)}.'

## Question of Understanding 

1. Are the values under the randomization inference loops (a result of `ri_sim_experiment`) changing? 
2. Should they be? Why or why not? 



## Randomization Inference Under A Strong Effect 

The previous time that we stepped through a series of randomiation inference loops, we had built the data so that there was *specifically* no effect. And, when we examined the distribution of the treatment effects that could have been possible under:  

- The observed data; 
- Presuming the sharp null was true; 

We saw that the distribution was centered at zero, and that a *lot* of the simulated distribution was more extreme than the treatment effect that we generated. 

In [ ]:
plt.hist(experiments_no_effect)
plt.title('The old example')

## Current Example 
Lets simulate these randomization inference loops again, this time with the 25 unit treatment effect. 

In [ ]:
experiments = [sim_experiment(df, NUMBER_PER_GROUP) for _ in range(1000)]

Before you plot this histogram: 

- Where do you think the histogram of randomization inference loops *should* be located? Should it be centered at either: 
    - Zero
    - 25
- Why? 
- Will this histogram have the same general *shape* as the previous histogram? Why or why not? 

In [ ]:
plt.hist(experiments)
plt.axvline(x=ate, color = 'darkorange')

Produce a statement of the test for how likely a treatment effect of this size is to occur under the presumption of the sharp null hypothesis? (This will be using the `p_value` function that we previously wrote. 

In [ ]:
p_value(ate=ate, ri_dist=experiments)